In [1]:
import pandas as pd
from transformers import pipeline
import os

In [4]:
articles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/InfoVis_FinalProject/nyt_cleaned.csv')
sentiment_pipeline = pipeline("sentiment-analysis")

<ipython-input-4-525597948cc4>:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  articles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/InfoVis_FinalProject/nyt_cleaned.csv')
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional t

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
output_dir = '/content/drive/MyDrive/Colab Notebooks/InfoVis_FinalProject'
csv_filename = os.path.join(output_dir, 'sentiment_nyt.csv')
tracking_file = os.path.join(output_dir,'tracking_txt')
error_csv = os.path.join(output_dir,'error_nyt.csv')

In [6]:
#Negative : -1 , POSITIVE: 1
#create new column in df with pos/neg and another with score
articles['sentiment'] = 0
articles['sentiment_score'] = 0.0
error_indices = []
error_log = []
chunk_size = 5000

try:
  with open(tracking_file, 'r') as f:
    last_row = int(f.read().strip())
except FileNotFoundError:
  last_row = 0

successful_rows = []
error_rows = []

for i,(index,row) in enumerate(articles.iloc[last_row:].iterrows(),start = last_row):
  try:
    sentiment = sentiment_pipeline(row['abstract'])[0]
    articles.at[index,'sentiment'] = 1 if sentiment['label'] == 'POSITIVE' else -1
    articles.at[index,'sentiment_score'] = sentiment['score']
    successful_rows.append(articles.loc[index])
  except Exception as e:
    # error_log.append(e)
    # print(f"An unexpected error occurred: {e}")
    # error_indices.extend(articles.index[articles['_id'].str.contains(row['_id'])].tolist())
    error_rows.append({'index': index, 'error': str(e)})
    print("errorINDEX:",index)
  if (i + 1) % chunk_size == 0 or i + 1 == len(articles):  # Save when chunk is complete or at the end
            # Save successful rows to the output CSV (append mode)
    if successful_rows:
      chunk_df = pd.DataFrame(successful_rows)
      chunk_df.to_csv(csv_filename, mode='a', header=(i == 0), index=False)
      successful_rows = []  # Clear the successful rows list

            # Save error rows to the error CSV (append mode)
    if error_rows:
      error_df = pd.DataFrame(error_rows)
      error_df.to_csv(error_csv, mode='a', header=(i == 0), index=False)
      error_rows = []  # Clear the error rows list

            # Update the tracking file with the last processed row index (successful and error rows included)
    with open(tracking_file, 'w') as f:
      f.write(str(i + 1))

# filename = 'nyt_sentiment.csv'
# df.to_csv(filename, index=False)

In [5]:
df = pd.read_csv(csv_filename)


<ipython-input-5-951bf33444da>:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_filename)


In [2]:
output_dir = '/content/drive/MyDrive/Colab Notebooks/InfoVis_FinalProject'
final_filename = os.path.join(output_dir, 'sentiment_nyt_clean.csv')

In [6]:
print("Shape:" ,df.shape)
df.drop_duplicates(inplace = True)
print("Shape after dropping duplicates:",df.shape)
# df.to_csv(final_filename,mode = 'a',header =(i ==0),index = False)



Shape: (1898485, 22)
Shape after dropping duplicates: (1898469, 22)


In [8]:
df.to_csv(final_filename,mode = 'a',header =(0),index = False)